In [2]:
import matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import warnings, os


import seaborn as sns
import scipy.stats as stats
import tensorflow as tf

import aggmap
from aggmap import loadmap, aggmodel


import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"

## fix random seed to get repeatale results
seed = 666
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
warnings.filterwarnings("ignore")


In [3]:
results_save_folder = './model_results/'
if not os.path.exists(results_save_folder):
    os.makedirs(results_save_folder)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() #(x_train, y_train), (x_test, y_test)
mnist_labels_dict = {0: 'T-shirt/top',1: 'Trouser',2: 'Pullover',
                     3: 'Dress',4: 'Coat', 5: 'Sandal', 
                     6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}

In [5]:
Y = pd.get_dummies(y_train).values.astype(float)
testY = pd.get_dummies(y_test).values.astype(float)

In [6]:
data_save_folder = '/raid/shenwanxiang/aggmap/fmnist/correlation/'

#orignal images
X_OR1 = load(os.path.join(data_save_folder, '01_X_OR1.data'))

#random shuffle
X_ORS1 = load(os.path.join(data_save_folder, '02_X_ORS1.data'))

#agg
X_ORSAgg1 = load(os.path.join(data_save_folder, '03_X_ORSAgg1.data'))

#cluster split channel
X_ORSAggC4 = load(os.path.join(data_save_folder, '04_X_ORSAggC4.data'))
X_ORSAggC10 = load(os.path.join(data_save_folder, '05_X_ORSAggC10.data'))

#self-defined channel
X_ORSAggV4 = load(os.path.join(data_save_folder, '06_X_ORSAggV4.data'))

In [7]:
fmp_types = [X_OR1, X_ORS1, X_ORSAgg1,  X_ORSAggC4,  X_ORSAggC10,  X_ORSAggV4]
fmp_names = ['OR1', 'ORS1','ORSAgg1',   'ORSAggC4',  'ORSAggC10', 'ORSAggV4']

In [ ]:
for name, x in zip(fmp_names, fmp_types) :
    
    X, testX = x[0]/255., x[1]/255.

    trainX, validX, trainY, validY = train_test_split(X, Y, test_size = 0.1, random_state = seed)

    epochs = 50
    patience = 1000
    dense_layers = [256, 128] #
    batch_size = 128
    lr = 1e-3

    dense_avf = 'relu'
    last_avf = 'softmax' #sigmoid in loss

    model = aggmodel.net.AggMapNet(trainX.shape[1:],
                                   n_outputs=trainY.shape[-1], 
                                   conv1_kernel_size = 3,
                                   dense_layers=dense_layers, 
                                   dense_avf = dense_avf, 
                                   last_avf=last_avf)

    opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #

    performance = aggmodel.cbks.CLA_EarlyStoppingAndPerformance((trainX, trainY), 
                                                                   (validX, validY), 
                                                                   patience = patience, 
                                                                   criteria = 'val_auc',
                                                                   last_avf= last_avf,
                                                                   verbose = 0)

    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

    history = model.fit(trainX, trainY, batch_size=batch_size,  # verbose= 2, 
                          epochs=epochs, shuffle = True, 
                          validation_data = (validX, validY), callbacks= [performance]) 

    train_loss, train_acc = model.evaluate(trainX, trainY, verbose=0)

    valid_loss, valid_acc = model.evaluate(validX, validY, verbose=0)

    test_loss, test_acc = model.evaluate(testX, testY, verbose=0)


    res = {'train_size':len(trainX),
           'valid_size': len(validX),
           'test_size': len(testX),
           'dense_layers':dense_layers,
           'dense_avf':dense_avf,
           'last_avf':last_avf, 
           'trainable_params':model.count_params(),
           'batch_size':batch_size,
           'lr':lr,
           'best_epochs_in_valid': performance.best_epoch + 1,
           
           'train_acc':train_acc,
           'valid_best_acc': valid_acc,
           'test_acc': test_acc,
           
           'train_loss':train_loss,
           'valid_best_loss': valid_loss,
           'test_loss': test_loss,}

    pd.DataFrame(history.history).to_csv(os.path.join(results_save_folder, 'training_%s.csv' % name))
    
    pd.DataFrame([res]).to_csv(os.path.join(results_save_folder, 'performance_%s.csv' % name))
    
    model.save(os.path.join(results_save_folder, 'model_%s.h5' % name))
    
    test_report = pd.DataFrame(classification_report(validY, np.round(model.predict(validX)),output_dict = True )).T
    test_report.to_csv(os.path.join(results_save_folder, 'test_report_%s.csv' % name))

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 10s 176us/sample - loss: 0.5676 - accuracy: 0.7905 - val_loss: 0.3585 - val_accuracy: 0.8652
Epoch 2/50
54000/54000 [==============================] - 6s 115us/sample - loss: 0.3300 - accuracy: 0.8797 - val_loss: 0.2907 - val_accuracy: 0.8952
Epoch 3/50
54000/54000 [==============================] - 6s 117us/sample - loss: 0.2767 - accuracy: 0.9001 - val_loss: 0.2757 - val_accuracy: 0.8987
Epoch 4/50
54000/54000 [==============================] - 6s 117us/sample - loss: 0.2419 - accuracy: 0.9116 - val_loss: 0.2383 - val_accuracy: 0.9125
Epoch 5/50
54000/54000 [==============================] - 6s 117us/sample - loss: 0.2221 - accuracy: 0.9186 - val_loss: 0.2464 - val_accuracy: 0.9033
Epoch 6/50
54000/54000 [==============================] - 7s 124us/sample - loss: 0.1988 - accuracy: 0.9264 - val_loss: 0.2247 - val_accuracy: 0.9175
Epoch 7/50
54000/54000 [==========================